Cross-Validation

In [18]:
# Imports
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold, LeaveOneOut
from sklearn.preprocessing import PolynomialFeatures
from OLS_Ridge import fit_polynomial_mod

In [ ]:
# Cross validation
def cross_validation(x, y, degrees=5, n_folds=5, LOO=False, ):
    
    # List to store results
    results = []

    if LOO:
        # Leave one out CV
        folds = LeaveOneOut()
    else:
        # Initiate k folds
        folds = KFold(n_folds)

    # Ensure x is 2D
    if x.ndim == 1:
        x = x.reshape(-1, 1)

    # Create polynomial features
    poly_features = PolynomialFeatures(degrees)
    
    # Split the date into n_folds
    for i, (train_indx, test_indx) in enumerate(folds.split(x)):
        # Extract training and testing data
        x_train, x_test, y_train, y_test = x[train_indx], x[test_indx], y[train_indx], y[test_indx]

        # Reshape data
        y_train = y_train.reshape(-1, 1)
        y_test = y_test.reshape(-1, 1)

        # Fit the training and test data
        X_train = poly_features.fit_transform(x_train)
        X_test = poly_features.transform(x_test)

        # Perform polynomial fit with selected method
        result = fit_polynomial_mod(X_train, X_test, y_train, y_test, n_iter=1000, momentum=True)

        # Store results
        results.append(result)

    return results
